In [30]:
import numpy as np
import pandas as pd

from glob import glob

In [60]:
year = 1970

In [62]:
df = pd.read_excel(
    "data/Chile/Elecciones Presidenciales 1952-1970.xlsx",
    sheet_name=f"Presidenciales {year}"
)
# 
df = df[~df.Comunas.isna()]
df = df.replace({"*": np.nan, "**": np.nan, "***": np.nan})
df = df.dropna(axis=1, how="all")

df = df.fillna(method="ffill")
print(list(df))
# df = df.rename(columns={"Nulos y blancos": "Nulos y Blancos"})
if year == 1952:
    df = df.drop(columns=["Total", "Total emitidos", "Nulos y blancos", "Total votos", "Inscritos"])
elif year == 1958:
    df = df.drop(columns=["Total ", "Nulos y Blancos", "Total votos"])
elif year == 1964:
    df = df.drop(columns=["Total", "Nulos", "Blancos", "Votos"])
elif year == 1970:
    df = df.drop(columns=["Total General", "Nulos y Blancos", "Votos"])

df.head()

['Provincia', 'Depto.', 'Comunas', 'Votos', 'Tomic', 'Alessandri', 'Allende', 'Nulos y Blancos', 'Total General']


,Provincia,Depto.,Comunas,Tomic,Alessandri,Allende
0,TARAPACA,Arica,ARICA,8407.0,7094.0,16598.0
1,TARAPACA,Arica,Belén,51.0,44.0,50.0
2,TARAPACA,Arica,Putre,71.0,48.0,84.0
3,TARAPACA,Arica,Codpa,106.0,31.0,21.0
4,TARAPACA,Arica,Gral. Lagos,27.0,44.0,29.0


In [63]:
df["polling_id"] = df["Provincia"] + "-" + df["Depto."] + "-" + df["Comunas"]

In [64]:
df = df.melt(id_vars=["Provincia", "Depto.", "Comunas", "polling_id"], var_name="candidate", value_name="value")

In [65]:
df["rank"] = df.groupby(["polling_id"])["value"].rank("min", ascending=False).astype(int)

In [66]:
tt = df.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0]).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt = tt[["polling_id", "candidate", "rate"]]

C:\Users\cnava\AppData\Local\Temp\ipykernel_24492\3993134105.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  tt["rate"] = tt.groupby(level=[0]).apply(lambda x: x/x.sum())


In [67]:
df = pd.merge(df, tt, on=["polling_id", "candidate"])

In [68]:
df.to_csv(f"data_output/Chile/{year}_first_round.csv.gz", compression="gzip", index=False)